# アンカリング効果の分析
架空の人々の架空のアンケート（人工データのこと）を基に，統計での信頼区間（CI:Confidence Interval）などを用いてアンカリング効果の分析方法を説明します。

## 箱ひげ図(box plot），エラーバー（error bar）などの関数の説明
seabornを用いる，これは，matplotlibはエラーバーに信頼区間が無いため。<br>
seaborn.boxplot : https://seaborn.pydata.org/generated/seaborn.boxplot.html<br>
　・最小，最大値を表現し，箱は下位より第一四分位数（25%），中央値（50%），第三四分位数（75%）を表す。<br>
　・外れ値（outliers）は，上記の説明より四分位数間（the inter-quartile rang）に基づき決定される。<br>
　・四分位数間の計算：https://en.wikipedia.org/wiki/Box_plot<br>
seaborn.barplot : https://seaborn.pydata.org/generated/seaborn.barplot.html<br>
　・データの平均値と信頼区間を表す。ディフォルトで95%信頼区間を用いる。<br>
seaborn.displot : https://seaborn.pydata.org/generated/seaborn.displot.html<br>
　・ヒストグラムとKDE(kernel density estimation)をプロットする。<br>
　・KDE <br>
 　　- https://ja.wikipedia.org/wiki/カーネル密度推定<br>
 　　- https://seaborn.pydata.org/generated/seaborn.kdeplot.html

In [ ]:
# -*- coding: utf-8 -*-
import numpy as np
import scipy.stats as stats

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

%matplotlib inline

#### 人工データの作成
二つのグループLとグループH，それぞれ100人ずつに，ある土地の降水量が前年の同月よりどれくらい増えたかの質問を行ったと想定。<br>
L(ow), H(igh)<br>
グループLには，「前年より3\%より増えたでしょうか，減ったでしょうか？　増減は何\% くらいでしょうか？」<br>
グループHには，「前年より5\%より増えたでしょうか，減ったでしょうか？　増減は何\% くらいでしょうか？」<br>
そのアンケート結果は人工的に生成。生成の仕方は，各グループは，各meanを中心に回答したとするデータ生成を行います。<br>
ただし，グループLは3\%は安すぎと考えてbiasLだけ上積みして回答したと想定（この仮説も様々に変更してください）。<br>
注意：<br>
・負の値は，前年より減ったとの回答とみなします。<br>
・正規分布を用い，標準偏差について，stdL < stdH とした意味は特にありません。<br>
これらについては，読者で様々な変更して確かめてください。

In [ ]:
np.random.seed(123)
num = 100
meanL, stdL = 3, 2.0
biasL = 0.5
meanH, stdH = 5, 3.0
xL = stats.norm.rvs(loc=meanL+biasL, scale=stdL, size=num)
xH = stats.norm.rvs(loc=meanH, scale=stdH, size=num)

### seabornはpandasのdataframeを使用
pandas.melt  https://pandas.pydata.org/docs/reference/api/pandas.melt.html<br>
はデータの格納形式を変更する

In [ ]:
df_tmp = pd.DataFrame({'L':xL, 'H':xH})
print(df_tmp.head())
df = df_tmp.melt(var_name='class', value_name='data')

ラベル：'class', 'data'を与えて，下記のように'class'がLとHで縦に並べる。

In [ ]:
df

#### Color Palettes
次を参照<br>
Choosing color palettes https://seaborn.pydata.org/tutorial/color_palettes.html<br>

In [ ]:
sns.set_theme(style='whitegrid', font_scale=1.5)

plt.figure(figsize=(7,4))
sns_boxplot = sns.boxplot(x="class", y="data", data=df, width=0.5, palette='pastel')

#図の保存は2つの2行を用いる
#figure = sns_boxplot.get_figure()
#figure.savefig('fig_SD_Anchoring_BoxPlot.pdf', bbox_inches='tight')

In [ ]:
plt.figure(figsize=(7,4))
sns_barplot = sns.barplot(x='class', y='data', data=df, capsize=0.2, errwidth=2.0, 
                          palette='pastel')

#図の保存は2つの2行を用いる
#figure = sns_barplot.get_figure()
#figure.savefig('fig_SD_Anchoring_BarPlot.pdf', bbox_inches='tight')

In [ ]:
plt.figure(figsize=(12,4))
sns_displot = sns.displot(data=df, x='data', hue='class', col='class', bins=range(-5, 15, 1), kde=True, palette='pastel')
sns_displot.set_ylabels('Frequency')

#plt.savefig('fig_SD_Anchoring_DisPlot.pdf', bbox_inches='tight')

In [ ]:
g = sns.displot(data=df, x='data', hue='class', bins=range(-5,15,1), element='step', kde=True, palette='pastel', height=4, aspect=1.5)
g.set_ylabels('Frequency', fontsize=20)

#### seaborn，binsのサイズ
ビンのサイズは重要なパラメータである。間違ったビンのサイズを使用すると、データの重要な特徴を不明瞭にする、ランダムな分散から見かけの特徴を作成するなど、誤解を招く可能性がある。デフォルトでは、displot() / histplot() は、データの分散とオブザベーションの数に基づき、デフォルトのビン・サイズを選択する。しかし、自動アプローチは、データの構造に関する特定の仮定に依存するため、過度に依存すべきではない。分布の印象が、異なるビン・サイズの間で一貫していることを常にチェックするべきである。サイズを直接選択するには、binwidthパラメータを設定する。<br>
参考URL：<br>
https://hawk-tech-blog.com/python-learn-seaborn-tutorial-plotting-function-distributions-of-data-1/<br>
https://hawk-tech-blog.com/python-learn-seaborn-tutorial-api-overview-function-overview-2/